# ✍️ Exercise: Intro to MLFlow - Part III

Now that we have loged models into MLFlow it's time to learn how register them and deploy them to a production environment.


- Load a regression dataset
- Train a model
- Log the model into MLFlow
- Register the model
- Stage the model into production/development
- Deploy the model using MLFlow

In this lesson, we will learn how to register a model. This is useful when, after logging multiple models, we want to select the one that has delivered the best results. We will work with the diabetes dataset provided by scikit-learn, which is a regression problem. 📝

In [1]:
from sklearn import datasets


# Download dataset and convert to pandas dataframe
diabetes_dataset = datasets.load_diabetes()
X = diabetes_dataset.data
y = diabetes_dataset.target

## Exercise I: Split the Data into Train and Test Sets ✂️

💡 Remember that we need to split our data into train and test sets. We can use the [`train_test_split` function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from `sklearn.model_selection` to do this. We should store the split into `X_train`, `y_train`, `X_test`, `y_test`.

In [2]:
from sklearn.model_selection import train_test_split


RANDOM_STATE = 42 
TEST_SIZE = 0.2 # 20% of the data will be used for testing

# 👇 Add the relevant code below to split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

## Exercise II: Train a Linear Regression Model 🤖

Then, train a [**linear regression model** using the scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

1. 👉 Initialize the model calling the `LinearRegression` class.
2. 👉 Train the model using the `fit` method.

In [3]:
from sklearn.linear_model import LinearRegression


# Add code to train the model 👇
model = LinearRegression() # Create a linear regression model
model.fit(X_train, y_train) # Train the model

LinearRegression()

## Exercise III: Compute the Accuracy of the Model 📊

Finally, compute the accuracy of the model using the [`mean_squared_error` function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) from the `sklearn.metrics` module.

1. 👉 Compute the predictions by passing the `X_test` to the `predict` method of the model.
2. 👉 Compute the accuracy using the `mean_squared_error` function and passing the `y_test` and the `predictions` as arguments.
3. 👉 Print the accuracy.

In [4]:
from sklearn.metrics import root_mean_squared_error


# Add code to calculate the mean squared error 👇
y_pred = model.predict(X_test) # Make predictions on the test set
rmse = root_mean_squared_error(y_test, y_pred) # Calculate the rmse
rmse

53.85344583676593

## Exercise IV: Create a Run and log the model and metrics. 🏃‍♂️📋

1. 👉 Connect to MLFlow
2. 👉 Set the experiment "Diabetes Linear Regression"

In [ ]:
import mlflow


EXPERIMENT_NAME = "Diabetes Linear Regression" # Name of the experiment in MLFlow
MLFLOW_TRACKING_URI = "http://localhost:5000" # MLFlow server URI


# Connect to MLFlow 👇
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI) # Connect to the MLFlow server
mlflow.set_experiment(EXPERIMENT_NAME) # Set the experiment name

<Experiment: artifact_location='mlflow-artifacts:/177159746396885082', creation_time=1737224616793, experiment_id='177159746396885082', last_update_time=1737224616793, lifecycle_stage='active', name='Diabetes Linear Regression', tags={}>


1. 👉 Log the root mean squared error metric using `mlflow.log_metric` function
2. 👉 Log the model using the `mlflow.sklearn.log_model` function.

In [ ]:
# launch a run to log the model
with mlflow.start_run() as run:
    
    # Add code to log the model, and the mean squared error 👇
    mlflow.log_metrics({"rmse": rmse}) # Log the rmse
    mlflow.sklearn.log_model(model, "model", input_example=X_test[:1]) # Log the model

2025/01/19 17:30:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-steed-638 at: http://localhost:5000/#/experiments/177159746396885082/runs/3f619765272242909f154725fe4cd90f.
2025/01/19 17:30:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/177159746396885082.


## Exercise V: Register the model 🗂️

Registering a model in MLFlow is a way to keep track of the different versions of the same model. Registered models have different versions that track changes in the model and allows

1. 👉 Get the **run ID** of the model you want to register using `run.info.run_id`.
2. 👉 Register the model using the `mlflow.register_model` function.

In [7]:
# register the model for this run
MODEL_NAME = "diabetes_prediction"  # change this to your model name


# Compute model path: models stored in a run follow this convention
model_path = f"runs:/{run.info.run_id}/model"  # fill the `run_id`` variable


# Register the model 
mlflow.register_model(model_path, MODEL_NAME)

Registered model 'diabetes_prediction' already exists. Creating a new version of this model...
2025/01/19 17:30:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: diabetes_prediction, version 2
Created version '2' of model 'diabetes_prediction'.


<ModelVersion: aliases=[], creation_timestamp=1737307853813, current_stage='None', description='', last_updated_timestamp=1737307853813, name='diabetes_prediction', run_id='3f619765272242909f154725fe4cd90f', run_link='', source='mlflow-artifacts:/177159746396885082/3f619765272242909f154725fe4cd90f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In this case I have registered my model manually.

![image.png](/workspaces/data-science-machine_learning_operations/data/registered_models.png)

## Exercise VI: Deploy a model 🚀

An API is like a "window" that allows us to interact with a model in a simple way. Essentially, it is an access point where we send data (usually in JSON format) and receive responses processed by the model within the API. We don’t need to understand how the model works internally, similar to driving a car: we turn the key to start it without understanding the engine.

Main elements of an API:
URL: An address like http://localhost:5000/invocations that points to the API.
Endpoint: A specific part of the URL that can vary depending on the desired function.
Method: The action we perform, such as retrieving, adding, modifying, or deleting data.
Body: Parameters or inputs sent to the model.
Header: Additional information included in the request.
Status Code: A response code indicating the result (e.g., 200 for success, 400 for error).

![image.jpg](/workspaces/data-science-machine_learning_operations/data/whats_an_api_02.jpg)

Deploying a model is a complex task that involves many steps. MLFlow simplifies this process by providing a set of tools to deploy models to different platforms. In this exercise, we will deploy a model to a local server. 

First, you need to connect the terminal to the MLFlow Server by setting the `MLFLOW_TRACKING_URI` environment variable. 

```bash
export MLFLOW_TRACKING_URI=http://localhost:5000
```

Then, you can deploy the model using the `mlflow models serve` command **in your terminal**:

```bash
mlflow models serve --model-uri models:/<model_name>/<model_version> --port 5001
```

Where `<model_name>` is the name of the model and `<model_version>` is the version of the model you want to deploy. You can find the name and version of the model in the MLFlow UI. Also the `--port` argument is the port where the server will be running. It's important to choose a port different than the `5000` port where the MLFlow server is running.

This is the code I used:

```bash
mlflow models serve --model-uri models:/diabetes_prediction/1 --env-manager conda --port 5001
```

## BONUS: Make a request to the model 📨

Finally, make a request to the model using the `requests` library. You can use the following code to make a request to the model:

In [8]:
# Define the URL and headers
URL = 'http://localhost:5001/invocations'
HEADERS = {'Content-Type': 'application/json'} # indicates the format of the data

Define the body of the request and in this case, it needs to be a dictionary. 

The vector length is 10, which can be verified in MLflow within the artifacts section. The message payload needs to be a JSON dictionary, where "inputs" serves as the key and "input_vector" as the value. Finally, we need to convert the format to a JSON string.

In [9]:
# Define the input payload
import json

input_vector = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]  # random input vector
payload = {'inputs': [input_vector]}  # wrap the input vector in a dictionary under the key 'inputs'
json_payload = json.dumps(payload)  # convert the payload to a JSON string

Send the POST request to the model

In [10]:
# Send the request
import requests

response = requests.post(URL, headers=HEADERS, data=json_payload)

Check the status code and the response of the request. If the status code is `200` the request was successful.

In [11]:
print(f"Status code: {response.status_code}")
print(f"Response body: {response.json()}")

Status code: 200
Response body: {'predictions': [1299.54418238401]}


Check that the model trained in the notebook generates the same predictions as the model deployed in the server.

In [12]:
model.predict([input_vector])

array([1299.54418238])